In [2]:
for i in range(10):
    ser.write(b'hello world')

NameError: name 'ser' is not defined

In [3]:
ser.write(b'hello world')

11

# Setup

In [3]:
import time
import serial
ser = serial.Serial(port='/dev/serial0',baudrate=9600,parity=serial.PARITY_NONE,stopbits=serial.STOPBITS_ONE,bytesize=serial.EIGHTBITS,timeout=1)

In [12]:
from enum import Enum

VERBOSITY_QUIET = 0
VERBOSITY_DEBUG = 1
Global_Verbosity = VERBOSITY_QUIET

def set_Verbosity(verbosity):
    Global_Verbosity = verbosity

def printLog(str, verbosity):
    if verbosity <= Global_Verbosity:
        print(str)

class eMsgType(Enum):
    MODE_CONTROL = b'\x10' #mode control
    COORDINATE = b'\x20' #enemy coordinate
    ACK_BIT = b'\x40' #always respond with the bit + this value of 40

class eSepChar(Enum): #start and ending hexes, acknowledgement bit
    STX_CHAR = b'\x02'
    ETX_CHAR = b'\x03'
    ACK = b'\x40'
    ACK_ASCII = b'\x41\x43\x4b' 
    UNUSED = b'\xFF'
#make it in b'', then send a byte array

class eAutoAimState(Enum):
    AUTO_AIM_INIT = 0 #turn on
    AUTO_AIM_IDLE = 1 #wait
    AUTO_AIM_SEND_COORDINATE = 2
    AUTO_AIM_WAIT_FOR_ACK = 3
    AUTO_AIM_SLEEP = 4 #turn off

class eReaderState(Enum):
    READ_STX = 0
    READ_PAYLOAD = 1
    READ_ERROR = 2
    
def getXthByte(target, byte_index): #xth byte from left
    byte_index=byte_index-1
    mask = 0xFF
    #print(hex((target >> (8 * byte_index)) & mask))
    return((target >> (8 * byte_index)) & mask).to_bytes(1,'big')

def send_coordinates(x,y):
	xlsb = getXthByte(x,1)
	xmsb = getXthByte(x,2)
	ylsb = getXthByte(y,1)
	ymsb = getXthByte(y,2)
	CoordinateData=eSepChar.STX_CHAR.value + eMsgType.COORDINATE.value + xlsb+xmsb+ylsb+ymsb+eSepChar.ETX_CHAR.value
	print("Printing coordinate data")
	print(CoordinateData)
	ser.write(CoordinateData) 

In [20]:
def AutoAim_Heartbeat(x,y,verbosity):
    #this is cv, arduino is control
	try: #try to print your current state. if not, reset to the init and read state
		printLog(AutoAim_Heartbeat.Heartbeat_State,VERBOSITY_DEBUG)
		printLog(AutoAim_Heartbeat.Reader_State,VERBOSITY_DEBUG)
	except:
		# main state, the state of the autoaim
		AutoAim_Heartbeat.Heartbeat_State = eAutoAimState.AUTO_AIM_INIT
		# sub state, your reader state
		AutoAim_Heartbeat.Reader_State = eReaderState.READ_STX
	
	if AutoAim_Heartbeat.Heartbeat_State == eAutoAimState.AUTO_AIM_INIT:
		ser.reset_input_buffer()
		ser.reset_output_buffer() #+ eMsgType.MODE_CONTROL.value + eSepChar.ACK_ASCII.value + eSepChar.ETX_CHAR.value)
		print("Reactor online. Sensors online. Weapons online. All systems nominal.\n")
		send_coordinates(x,y)#send coordinates
		AutoAim_Heartbeat.Heartbeat_State = eAutoAimState.AUTO_AIM_IDLE;#change to idle mode
	elif AutoAim_Heartbeat.Heartbeat_State == eAutoAimState.AUTO_AIM_IDLE:#if in idle mode
		#polling for control msg, if auto aim mode start msg received, ACK back
        #where should the no consecutive timer be implemented? 
		if ser.inWaiting()>0:
			if AutoAim_Heartbeat.Reader_State == eReaderState.READ_STX: #if you are reading STX
				if ser.read(1) == eSepChar.STX_CHAR.value: #check if the first bit is the STX value
					AutoAim_Heartbeat.Reader_State = eReaderState.READ_PAYLOAD	#if yes, enter reading state
			elif AutoAim_Heartbeat.Reader_State == eReaderState.READ_PAYLOAD: #if in reading state
				if ser.read(1) == eMsgType.MODE_CONTROL.value: #check if the bit you are recieving is telling you to change mode
                    #does ser.read read the first bit, or 1 of the next bit? fix this
                    #read number probably wrong here, update it
					newState = int(ser.read(1)) #read the next bit
					ACKData = eSepChar.STX_CHAR.value + eSepChar.ACK.value + eSepChar.ACK_ASCII.value + eSepChar.UNUSED.value + eSepChar.ETX_CHAR.value
					if newState==eAutoAimState.AUTO_AIM_INIT: #switch case, swap based on the bit that was read
						AutoAim_Heartbeat.Heartbeat_State = eAutoAimState.AUTO_AIM_INIT
						print("I am changing my mode to initialization\n") #send acknowledgement 
						print(ACKData)
						ser.write(ACKData)
					elif newState==eAutoAimState.AUTO_AIM_IDLE:                  
						AutoAim_Heartbeat.Heartbeat_State = eAutoAimState.AUTO_AIM_IDLE
						print("I am changing my mode to idle\n") #send acknowledgement 
						print(ACKData)
						ser.write(ACKData)
					elif newState==eAutoAimState.AUTO_AIM_SLEEP:
						AutoAim_Heartbeat.Heartbeat_State = eAutoAimState.AUTO_AIM_SLEEP
						print("I am changing my mode to waiting for sleep\n") #send acknowledgement
						print(ACKData)
						ser.write(ACKData)
				else:
					AutoAim_Heartbeat.Reader_State = eReaderState.READ_ERROR #read something wrong, throw error
			else:
				printLog("Unexpected AutoAim API state!")
				raise RuntimeError
		#cc = ser.read(ser.inWaiting())
		#print(cc.decode())
		#print(cc)
	elif AutoAim_Heartbeat.Heartbeat_State == eAutoAimState.AUTO_AIM_SLEEP:
	    #send acknowledgement
		ser.write(eSepChar.STX_CHAR + eMsgType.MODE_CONTROL + eSepChar.ACK_ASCII + eSepChar.ETX_CHAR ) 
		print("Auto aim disabled.")
	else:
		printLog("Unexpected AutoAim API state!")
		raise RuntimeError
	# CV should turn off if state is not active to save battery
	fCvActive = not (AutoAim_Heartbeat.Heartbeat_State in (eAutoAimState.AUTO_AIM_IDLE,eAutoAimState.AUTO_AIM_SLEEP))
	return fCvActive

#for the sake of testing 
AutoAim_Heartbeat(69,42,1)

Reactor online. Sensors online. Weapons online. All systems nominal.

Printing coordinate data
b'\x02 E\x00*\x00\x03'


False

b'i'

In [21]:
import struct
def AutoAim_Heartbeat(x,y,verbosity):
	# setup static variable
    #this is cv, arduino is control
	try: #is this backwards? i need to review try except statements
		printLog(AutoAim_Heartbeat.Heartbeat_State,VERBOSITY_DEBUG)
		printLog(AutoAim_Heartbeat.Reader_State,VERBOSITY_DEBUG)
	except:
		# main state, the state of the autoaim
		AutoAim_Heartbeat.Heartbeat_State = eAutoAimState.AUTO_AIM_INIT
		# sub state, your reader state
		AutoAim_Heartbeat.Reader_State = eReaderState.READ_STX
	
	if AutoAim_Heartbeat.Heartbeat_State == eAutoAimState.AUTO_AIM_INIT:
		ser.reset_input_buffer()
		ser.reset_output_buffer() #+ eMsgType.MODE_CONTROL.value + eSepChar.ACK_ASCII.value + eSepChar.ETX_CHAR.value)
		ACKData = eSepChar.STX_CHAR.value + eSepChar.ACK.value + eSepChar.ACK_ASCII.value + eSepChar.UNUSED.value + eSepChar.ETX_CHAR.value
		print(ACKData)
		ser.write(ACKData)
#for the sake of testing 
AutoAim_Heartbeat(10,10,1)


b'\x02@ACK\xff\x03'


## Unit test

In [23]:
AutoAim_Heartbeat(10,10,1)

b'\x02@ACK\xff\x03'
